In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [2]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [3]:
# Read data
def get_data_by_store(state_cat):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    df['state_cat']=df['state_id'].astype(str) + df['cat_id'].astype(str) 
    
    # Leave only relevant store
    df = df[df['state_cat']==state_cat]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for state_cat in STA_CATS:
        temp_df = pd.read_pickle('test_'+state_cat+'.pkl')
        temp_df['state_cat'] = state_cat
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]

In [4]:
def make_lag_roll(LAG_DAY,lag_df_new):
   
    lag_df = base_test[['id','d',TARGET]]
    
    lag_df=lag_df.sort_values(by=["d"])
  
    for i in range(0,len(LAG_DAY)):

        shift_day = LAG_DAY[i][0]
        roll_wind = LAG_DAY[i][1]
        col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
        lag_df[col_name] = (lag_df.groupby(['id'])[TARGET]).transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
        

    lag_df_new=lag_df.drop(columns=["sales"])
    return lag_df_new

In [5]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    "lambda":0.1,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 



# lgb_params ={
#         "objective" : "tweedie",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
#         "metric": ["rmse"],
#         "nthread": -1,
#         "tweedie_variance_power":1.1,
#     'verbosity': 1,
# #     'num_iterations' : 1500,
#     'num_leaves': 128,
#     "min_data_in_leaf": 104,
#     }




# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 

# 'max_bin': 100
## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [6]:
VER = 1                          # Our model version
SEED = 888                     # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','date','wm_yr_wk','d',TARGET,'state_cat','state_id','cat_id']
mean_features   = ['enc_store_id_mean','enc_store_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STA_CATS=['CAFOODS','TXFOODS','CAHOUSEHOLD','WIFOODS','TXHOUSEHOLD',
          'WIHOUSEHOLD','CAHOBBIES','WIHOBBIES','TXHOBBIES']

#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,28,56]:
        ROLS_SPLIT.append([i,j])

In [7]:
for state_cat in STA_CATS:
    print('Train', state_cat)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(state_cat)
    
    print(features_columns)
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    X_valid=grid_df[valid_mask][features_columns]
    
    valid_pred = grid_df[valid_mask].reset_index(drop=True)
    cols = [col for col in ['id', 'd']]
    valid_pred = valid_pred[cols]
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+state_cat+'.pkl')
    del grid_df
    gc.collect()
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    valid_pred['pred']=estimator.predict(X_valid)
    valid_pred.to_pickle('valid_pred_'+state_cat+'.pkl')
    del valid_pred
    gc.collect()
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+state_cat+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    #     !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CAFOODS
['item_id', 'dept_id', 'store_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_store_id_mean', 'enc_store_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7

[100]	valid_0's rmse: 1.28427
[200]	valid_0's rmse: 1.27759
[300]	valid_0's rmse: 1.27295
[400]	valid_0's rmse: 1.26931
[500]	valid_0's rmse: 1.26594
[600]	valid_0's rmse: 1.26283
[700]	valid_0's rmse: 1.26014
[800]	valid_0's rmse: 1.25774
[900]	valid_0's rmse: 1.25547
[1000]	valid_0's rmse: 1.2531
[1100]	valid_0's rmse: 1.25059
[1200]	valid_0's rmse: 1.2478
[1300]	valid_0's rmse: 1.24546
[1400]	valid_0's rmse: 1.24312
Train WIHOUSEHOLD
['item_id', 'dept_id', 'store_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', '

In [8]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day


for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    
    
    lag_df_new = pd.DataFrame()

    lag_df_new=make_lag_roll(ROLS_SPLIT,lag_df_new)


    grid_df =  grid_df.merge(lag_df_new, on=['id','d'], how='left')


    for state_cat in STA_CATS:

        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+state_cat+'_v'+str(VER)+'.bin' 

        estimator = pickle.load(open(model_path, 'rb'))

        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        state_cat_mask = base_test['state_cat']==state_cat

        mask = (day_mask)&(state_cat_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()

    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    
    del temp_df, lag_df_new

all_preds = all_preds.reset_index(drop=True)
all_preds.head()

Predict | Day: 1
##########  3.71 min round |  3.71 min total |  37360.85 day sales |
Predict | Day: 2
##########  3.71 min round |  7.42 min total |  35366.06 day sales |
Predict | Day: 3
##########  3.72 min round |  11.14 min total |  35219.50 day sales |
Predict | Day: 4
##########  3.71 min round |  14.86 min total |  35475.64 day sales |
Predict | Day: 5
##########  3.72 min round |  18.58 min total |  42097.20 day sales |
Predict | Day: 6
##########  3.73 min round |  22.30 min total |  50644.58 day sales |
Predict | Day: 7
##########  3.72 min round |  26.02 min total |  53402.41 day sales |
Predict | Day: 8
##########  3.72 min round |  29.74 min total |  44249.42 day sales |
Predict | Day: 9
##########  3.72 min round |  33.46 min total |  43996.29 day sales |
Predict | Day: 10
##########  3.71 min round |  37.18 min total |  38597.76 day sales |
Predict | Day: 11
##########  3.71 min round |  40.89 min total |  40830.67 day sales |
Predict | Day: 12
##########  3.73 min roun

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.756095,0.760966,0.731943,0.745309,1.013746,0.967076,1.063141,0.981059,0.864893,...,0.836363,1.095822,1.069486,0.857150,0.779583,0.772847,0.679959,0.823186,1.030301,0.972071
1,FOODS_1_002_CA_1_validation,0.459146,0.451144,0.431361,0.391276,0.513431,0.623723,0.540597,0.443445,0.484276,...,0.424325,0.562498,0.509234,0.424400,0.390300,0.414316,0.433981,0.466368,0.549635,0.510582
2,FOODS_1_003_CA_1_validation,0.828840,0.660785,0.751772,0.627396,0.770960,0.962133,0.887442,0.768919,0.766623,...,0.766825,0.837034,0.945565,0.741509,0.710926,0.654792,0.652979,0.765017,1.023400,0.913067
3,FOODS_1_004_CA_1_validation,0.042049,2.844217,7.315287,7.566521,10.315833,11.055734,12.289018,8.944695,8.732646,...,10.253500,11.576838,11.222777,8.128686,7.006207,7.169784,7.615045,9.910253,11.441161,10.356463
4,FOODS_1_005_CA_1_validation,1.340218,1.278595,1.386443,1.176829,1.408251,1.829454,1.750965,1.209186,1.166439,...,1.366892,1.525246,1.537361,1.169851,1.062878,1.002113,1.171353,1.335236,1.667461,1.615660


In [9]:
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv('sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('lgb_bystcat_fe6.csv', index=False) #0.47429